### Luxury Hospitality Landscape Labor ML Based Forecasting Workflow

- Business Objective: 

    - Predict weekly landscape labor demand by location and season to proactively prevent under-staffing during high guest exposure periods while minimizing overtime risk.

- Business Questions:
    - Where are labor hotspots likely to emerge next week?
    - When does predicted demand exceed normal staffing capacity?
    - Which locations carry the highest operational risk if understaffed?
    - What action should ops managers take before the risk happens?

### PHASE 1- Data Foundation

This phase aims to understand how labor demand is distributed across landscape placements and what operational drivers are associated with higher workload.

Load & Normalize

In [1]:
import pandas as pd

op_fact = pd.read_csv("Operational_fact.csv")
area_dim = pd.read_csv("Area_dim.csv")
var_fact = pd.read_csv("Variety_fact.csv")

# drop junk index column if exists
op_fact = op_fact.drop(columns=["Unnamed: 0"], errors="ignore")
var_fact = var_fact.drop(columns=["Unnamed: 0"], errors="ignore")

# ensure date is datetime
op_fact["date"] = pd.to_datetime(op_fact["date"])

In [3]:
# Clean + convert
op_fact["hours"] = (
    op_fact["hours"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

op_fact["hours"] = pd.to_numeric(
    op_fact["hours"],
    errors="coerce"
)

op_fact["hours"] = op_fact["hours"].abs()


Week Key

In [4]:
op_fact["week"] = (
    op_fact["date"].dt.isocalendar().year.astype(str)
    + "-W"
    + op_fact["date"].dt.isocalendar().week.astype(str).str.zfill(2)
)

Task Hours

In [5]:
task_weekly = (
    op_fact
    .groupby(["week", "place_name_id", "task_id"], as_index=False)
    .agg(task_hours=("hours", "sum"))
)

Task Ratios

In [6]:
total_hours = (
    task_weekly
    .groupby(["week", "place_name_id"], as_index=False)
    .agg(total_hours=("task_hours", "sum"))
)

task_weekly = task_weekly.merge(
    total_hours,
    on=["week", "place_name_id"],
    how="left"
)

task_weekly["task_ratio"] = (
    task_weekly["task_hours"] / task_weekly["total_hours"]
)

In [7]:
task_mix = (
    op_fact
    .groupby(["place_name_id", "week", "task_id"])
    .agg(task_hours=("hours", "sum"))
    .reset_index()
)

task_mix["task_ratio"] = (
    task_mix["task_hours"] /
    task_mix.groupby(["place_name_id", "week"])["task_hours"].transform("sum")
)


Pivot Task Features

In [8]:
task_features = (
    task_weekly
    .pivot_table(
        index=["week", "place_name_id"],
        columns="task_id",
        values=["task_hours", "task_ratio"],
        fill_value=0
    )
)

task_features.columns = [
    f"{metric}_task_{task}"
    for metric, task in task_features.columns
]

task_features = task_features.reset_index()

LAG & VOLATILITY FEATURES

In [9]:
task_volatility = (
    op_fact
    .groupby(["week", "place_name_id"], as_index=False)
    .agg(task_hours_std=("hours", "std"))
)

task_volatility["task_hours_std"] = (
    task_volatility["task_hours_std"].fillna(0)
)

Season Control

In [10]:
season_weekly = (
    op_fact
    .groupby(["week", "place_name_id"], as_index=False)
    .agg(
        season_id=("season_id", lambda x: x.mode().iloc[0])
    )
)

In [11]:
weekly_ops = (
    task_features
    .merge(task_volatility, on=["week", "place_name_id"], how="left")
    .merge(season_weekly, on=["week", "place_name_id"], how="left")
)

In [12]:
op_fact["week_id"] = (
    op_fact["date"].dt.isocalendar().year.astype(str)
    + "-W"
    + op_fact["date"].dt.isocalendar().week.astype(str).str.zfill(2)
)

In [13]:
def safe_mode(x):
    m = x.mode()
    return m.iloc[0] if len(m) > 0 else None


weekly_fact = (
    op_fact
    .groupby(
        ["week_id", "place_name_id"],
        as_index=False
    )
    .agg(
        weekly_hours=("hours", "sum"),                     # FIX: use hours
        task_diversity=("task_id", "nunique"),
        plant_diversity=("plant_variety_id", "nunique"),
        display_diversity=("display_type_id", "nunique"),
        active_days=("date", "nunique"),
        dominant_season=("season_id", safe_mode)            # FIX: safe mode
    )
)

In [14]:
# DAILY TOTAL HOURS
daily_totals = (
    op_fact
    .groupby(["place_name_id", "date"])
    .agg(
        daily_hours=("hours", "sum"),
        season_id=("season_id", lambda x: x.mode()[0])
    )
    .reset_index()
)

In [16]:
# CLEAN & VALIDATE PLACE DIMENSION
area_dim = (
    area_dim[
        ["place_name_id", "place_name", "foot_traffic_score"]
    ]
    .drop_duplicates(subset=["place_name_id"])
    .reset_index(drop=True)
)

In [17]:
df = (
    op_fact
    .merge(area_dim, on="place_name_id", how="left")
    .merge(var_fact, on="plant_variety_id", how="left")
)

### PHASE 2- Feature Engineering

Objective: Build forecast-quality models that predict weekly operational load per area.

Target : weekly_hours
Grain : week_id × place_name_id

In [18]:
model_df = (
    weekly_fact
    .merge(
        area_dim,
        on="place_name_id",
        how="left",
        validate="many_to_one"
    )
)

In [19]:
model_df["foot_traffic_score"] = model_df["foot_traffic_score"].fillna(
    model_df["foot_traffic_score"].median()
)

In [20]:
# Target
y = model_df["weekly_hours"]

# Features
feature_cols = [
    "task_diversity",
    "plant_diversity",
    "display_diversity",
    "active_days",
    "foot_traffic_score"
]

X = model_df[feature_cols]

In [21]:
# Time-Aware Train
model_df = model_df.sort_values("week_id")

split_idx = int(len(model_df) * 0.8)

X_train = X.iloc[:split_idx]
X_val   = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_val   = y.iloc[split_idx:]

In [22]:
model_df.duplicated(["week_id", "place_name_id"]).sum() == 0
model_df.isna().mean().max() < 0.05

True

In [23]:
model_df.columns

Index(['week_id', 'place_name_id', 'weekly_hours', 'task_diversity',
       'plant_diversity', 'display_diversity', 'active_days',
       'dominant_season', 'place_name', 'foot_traffic_score'],
      dtype='object')

### PHASE 3- Predictive Core

In [24]:
TARGET = "weekly_hours"

EXCLUDE_COLS = [
    "weekly_hours",
    "week_id",
    "place_name_id",
    "place_name"   # label, not signal
]

FEATURES = [c for c in model_df.columns if c not in EXCLUDE_COLS]

X = model_df[FEATURES]
y = model_df[TARGET]

In [25]:
# Ensure correct ordering
model_df = model_df.sort_values("week_id").reset_index(drop=True)

unique_weeks = model_df["week_id"].unique()
cutoff_week = unique_weeks[int(len(unique_weeks) * 0.8)]

train_idx = model_df["week_id"] < cutoff_week
val_idx   = model_df["week_id"] >= cutoff_week

X_train, X_val = X.loc[train_idx], X.loc[val_idx]
y_train, y_val = y.loc[train_idx], y.loc[val_idx]

Guest Exposure

In [26]:
df = model_df.copy()
df["guest_exposure_score"] = df["foot_traffic_score"]

Labor Intensity

In [27]:
df["labor_intensity"] = df["weekly_hours"] / df["active_days"]

Biological Sensitivity

In [28]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

bio_components = df[
    ["plant_diversity", "display_diversity", "task_diversity"]
]

bio_scaled = scaler.fit_transform(bio_components)

df[["bio_plant", "bio_display", "bio_task"]] = bio_scaled


In [29]:
df["biological_sensitivity_score"] = (
    0.4 * df["bio_plant"] +
    0.3 * df["bio_display"] +
    0.3 * df["bio_task"]
)

In [30]:
df[
    ["guest_exposure_score",
     "biological_sensitivity_score",
     "labor_intensity"]
].describe()

,guest_exposure_score,biological_sensitivity_score,labor_intensity
count,832.000000,832.000000,832.000000
mean,5.128606,0.132452,1.754112
std,2.590188,0.171710,0.698643
min,2.000000,0.000000,0.250000
25%,2.000000,0.000000,1.333333
50%,5.000000,0.057143,1.510000
75%,7.000000,0.264286,2.060000
max,10.000000,1.000000,9.380000


Define Quantile Thresholds

In [31]:
exp_q = df["guest_exposure_score"].quantile([0.33, 0.66])
bio_q = df["biological_sensitivity_score"].quantile([0.33, 0.66])
lab_q = df["labor_intensity"].quantile([0.33, 0.66])

Bucketization Helper

In [32]:
def level(x, q1, q2):
    if x >= q2:
        return "High"
    elif x >= q1:
        return "Medium"
    else:
        return "Low"

Create Levels

In [33]:
df["Exposure_Level"] = df["guest_exposure_score"].apply(
    lambda x: level(x, exp_q.loc[0.33], exp_q.loc[0.66])
)

df["Bio_Level"] = df["biological_sensitivity_score"].apply(
    lambda x: level(x, bio_q.loc[0.33], bio_q.loc[0.66])
)

df["Labor_Level"] = df["labor_intensity"].apply(
    lambda x: level(x, lab_q.loc[0.33], lab_q.loc[0.66])
)

Segment Logic

In [34]:
def assign_segment(row):

    if (
        row["Exposure_Level"] == "High"
        and row["Bio_Level"] == "High"
        and row["Labor_Level"] in ["Low", "Medium"]
    ):
        return "Efficient Staples"

    if (
        row["Bio_Level"] == "Low"
        and row["Labor_Level"] == "High"
    ):
        return "Iconic Zones"

    if (
        row["Exposure_Level"] == "High"
        and row["Bio_Level"] == "High"
        and row["Labor_Level"] == "High"
    ):
        return "Silent Drains"

    return "Other / Mixed"


In [35]:
df["Operational_Segment"] = df.apply(assign_segment, axis=1)

Segment Validation Test

In [36]:
segment_summary = (
    df.groupby("Operational_Segment")
      .agg(
          avg_exposure=("guest_exposure_score", "mean"),
          avg_bio=("biological_sensitivity_score", "mean"),
          avg_labor=("labor_intensity", "mean"),
          zones=("place_name_id", "nunique")
      )
      .round(2)
      .reset_index()
)

segment_summary

,Operational_Segment,avg_exposure,avg_bio,avg_labor,zones
0,Efficient Staples,7.45,0.27,1.47,10
1,Other / Mixed,4.27,0.06,1.67,23
2,Silent Drains,7.38,0.35,2.47,10


In [37]:
phase3_output = df[
    [
        "week_id",
        "place_name_id",
        "place_name",
        "guest_exposure_score",
        "biological_sensitivity_score",
        "labor_intensity",
        "Operational_Segment"
    ]
].copy()


In [38]:
assert phase3_output.isna().sum().sum() == 0, "❌ NaNs detected in Phase 3 output"
assert phase3_output["Operational_Segment"].isin(
    ["Efficient Staples", "Iconic Zones", "Silent Drains", "Other / Mixed"]
).all(), "❌ Unknown segment label detected"

In [39]:
# Summary - Phase 3
phase3_summary = (
    phase3_output
    .groupby("Operational_Segment")
    .agg(
        zones=("place_name_id", "nunique"),
        avg_exposure=("guest_exposure_score", "mean"),
        avg_bio=("biological_sensitivity_score", "mean"),
        avg_labor=("labor_intensity", "mean")
    )
    .round(2)
    .reset_index()
)

phase3_summary


,Operational_Segment,zones,avg_exposure,avg_bio,avg_labor
0,Efficient Staples,10,7.45,0.27,1.47
1,Other / Mixed,23,4.27,0.06,1.67
2,Silent Drains,10,7.38,0.35,2.47


Segment Summary

Efficient Staples (10 zones)
High guest exposure with controlled labor intensity, despite moderate biological complexity. These areas demonstrate that visible, biologically demanding landscapes can be maintained efficiently through good design and stable routines. They should serve as operational benchmarks for replication.

Silent Drains (10 zones)
Equally high guest exposure but significantly higher labor intensity, driven by greater biological sensitivity. These zones consume disproportionate resources and pose the highest risk to guest experience if performance slips. They require priority monitoring and protected budgets.

Other / Mixed (23 zones)
Lower exposure and minimal biological complexity with moderate labor demand. These areas are operationally stable and not critical for immediate intervention, making them lower-priority in strategic planning.

Bottomline: 

Differences in labor demand are driven more by plant and display design than by location or guest exposure. Efficient Staples show what good design looks like, while Silent Drains highlight where focused investment and risk management are essential.

#### Baseline Model: Rain Forest

Why RF?

- Handles non-linearity
- Stable baseline
- Robust to feature noise

In [76]:
# Baseline Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

rf_pred = rf.predict(X_val)

mae_rf = mean_absolute_error(y_val, rf_pred)
rmse_rf = np.sqrt(mean_squared_error(y_val, rf_pred))

mae_rf, rmse_rf


(0.7378103112691218, np.float64(1.1167767186073163))

A baseline Random Forest model predicts weekly labor demand with a mean absolute error of approximately 0.74 hours and an RMSE of approximately 1.1 hours, demonstrating that weekly labor demand is meaningfully predictable from operational data.

#### 4.1 Segment Contrast Diagnostics

In [77]:
segment_diag = (
    phase3_output
    .groupby("Operational_Segment")
    .agg(
        avg_labor=("labor_intensity", "mean"),
        avg_exposure=("guest_exposure_score", "mean"),
        avg_bio=("biological_sensitivity_score", "mean"),
        zones=("place_name_id", "nunique")
    )
    .round(2)
    .reset_index()
)

segment_diag

,Operational_Segment,avg_labor,avg_exposure,avg_bio,zones
0,Efficient Staples,1.39,7.44,0.28,10
1,Other / Mixed,1.63,4.27,0.06,23
2,Silent Drains,2.45,7.38,0.35,10


#### 4.2 Interaction Effect Check

In [78]:
phase3_output["Exposure_Bio_Interaction"] = (
    phase3_output["guest_exposure_score"]
    * phase3_output["biological_sensitivity_score"]
)

In [79]:
interaction_summary = (
    phase3_output
    .groupby("Operational_Segment")
    .agg(
        avg_interaction=("Exposure_Bio_Interaction", "mean"),
        avg_labor=("labor_intensity", "mean")
    )
    .round(2)
    .reset_index()
)

interaction_summary


,Operational_Segment,avg_interaction,avg_labor
0,Efficient Staples,2.02,1.39
1,Other / Mixed,0.20,1.63
2,Silent Drains,2.56,2.45


High interaction + high labor → true risk zone

#### 4.3 Residual Stress Test (Within Segment)

In [80]:
segment_variance = (
    phase3_output
    .groupby("Operational_Segment")
    .agg(
        labor_std=("labor_intensity", "std"),
        labor_p90=("labor_intensity", lambda x: x.quantile(0.9))
    )
    .round(2)
    .reset_index()
)

segment_variance

,Operational_Segment,labor_std,labor_p90
0,Efficient Staples,0.42,1.73
1,Other / Mixed,0.71,2.41
2,Silent Drains,0.61,3.00


### PHASE 4- Model Deepening (Diagnostics + Stability)

Ensure that downstream decisions are based on stable, trustworthy drivers, and clearly flag where decisions must be risk-adjusted or scenario-based.

#### 4.1 Enrich Weekly Facts with Exposure

In [86]:
weekly_fact.columns


Index(['week_id', 'place_name_id', 'weekly_hours', 'task_diversity',
       'plant_diversity', 'display_diversity', 'active_days',
       'dominant_season'],
      dtype='object')

In [107]:
# Build exposure lookup (static dimension)
area_exposure = (
    area_dim[
        ["place_name_id", "foot_traffic_score"]
    ]
    .drop_duplicates()
)

# Join to weekly_fact
weekly_fact_enriched = weekly_fact.merge(
    area_exposure,
    on="place_name_id",
    how="left",
    validate="many_to_one"
)

In [108]:
assert "foot_traffic_score" in weekly_fact_enriched.columns

#### 4.2 Exposure × Operational Interaction Signals

In [109]:
interaction_features = [
    "task_diversity",
    "plant_diversity",
    "display_diversity",
    "active_days"
]

for col in interaction_features:
    weekly_fact_enriched[f"{col}_x_exposure"] = (
        weekly_fact_enriched[col]
        * weekly_fact_enriched["foot_traffic_score"]
    )

#### 4.3 Rolling-Time Stability Test (WITH Interaction)

In [110]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

FEATURES = (
    interaction_features
    + ["foot_traffic_score"]
    + [f"{c}_x_exposure" for c in interaction_features]
)

feature_importance_history = []

weeks = weekly_fact_enriched["week_id"].sort_values().unique()
cutoffs = np.linspace(0.6, 0.8, 4)

for c in cutoffs:
    split_point = weeks[int(len(weeks) * c)]
    
    train_df = weekly_fact_enriched[
        weekly_fact_enriched["week_id"] <= split_point
    ]
    
    X_train = train_df[FEATURES]
    y_train = train_df["weekly_hours"]
    
    rf_stability = RandomForestRegressor(
        n_estimators=300,
        max_depth=14,
        min_samples_leaf=4,
        random_state=42,
        n_jobs=-1
    )
    
    rf_stability.fit(X_train, y_train)
    
    fi_tmp = pd.DataFrame({
        "feature": FEATURES,
        "importance": rf_stability.feature_importances_,
        "cutoff_week_id": split_point
    })
    
    feature_importance_history.append(fi_tmp)

fi_stability = pd.concat(feature_importance_history, ignore_index=True)


#### 4.4 Normalize Importance

In [111]:
fi_stability["importance_norm"] = (
    fi_stability["importance"]
    / fi_stability.groupby("cutoff_week_id")["importance"].transform("sum")
)

#### 4.5 Interpretive Aggregation

In [112]:
fi_summary = (
    fi_stability
    .groupby("feature")
    .agg(
        avg_importance=("importance_norm", "mean"),
        std_importance=("importance_norm", "std")
    )
    .sort_values("avg_importance", ascending=False)
    .round(4)
    .reset_index()
)

fi_summary

,feature,avg_importance,std_importance
0,plant_diversity,0.4916,0.1250
1,active_days,0.3967,0.1394
2,task_diversity_x_exposure,0.0260,0.0035
3,active_days_x_exposure,0.0236,0.0034
4,display_diversity_x_exposure,0.0213,0.0015
5,plant_diversity_x_exposure,0.0179,0.0073
6,foot_traffic_score,0.0086,0.0021
7,task_diversity,0.0073,0.0012
8,display_diversity,0.0069,0.0010


- High avg + low std → structural driver (trusted)
- High avg + high std → seasonal amplifier (risk-sensitive)
- Low avg → safe to ignore in decision logic
==========================================================================
- plant_diversity_x_exposure
→ dominant in Silent Drains
- task_diversity_x_exposure
→ unstable drivers (manual complexity)
- active_days
→ baseline load, not risk

Silent Drains are risky because complex operations are amplified by visibility, not because of random inefficiency.

#### C. PREDICTION FRAGILITY (UNCERTAINTY)
Key question: When interactions are strong, do trees disagree more?

##### Compute Tree-Level Prediction Uncertainty

In [114]:
import numpy as np

def rf_prediction_uncertainty(rf_model, X):
    """
    Returns mean prediction and std deviation across trees
    """
    tree_preds = np.array([
        tree.predict(X)
        for tree in rf_model.estimators_
    ])
    
    return tree_preds.mean(axis=0), tree_preds.std(axis=0)

##### Fit Stability RF on FULL Phase-4 Feature Set

In [115]:
FEATURES = (
    interaction_features
    + ["foot_traffic_score"]
    + [f"{c}_x_exposure" for c in interaction_features]
)

rf_fragility = RandomForestRegressor(
    n_estimators=300,
    max_depth=14,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

rf_fragility.fit(
    weekly_fact_enriched[FEATURES],
    weekly_fact_enriched["weekly_hours"]
)

RandomForestRegressor(max_depth=14, min_samples_leaf=4, n_estimators=300,
                      n_jobs=-1, random_state=42)

##### Generate Fragility Signals

In [116]:
mean_pred, pred_uncertainty = rf_prediction_uncertainty(
    rf_fragility,
    weekly_fact_enriched[FEATURES]
)

weekly_fact_enriched["predicted_hours"] = mean_pred
weekly_fact_enriched["prediction_uncertainty"] = pred_uncertainty

c:\Users\sinta\OneDrive\Lampiran\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\sinta\OneDrive\Lampiran\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\sinta\OneDrive\Lampiran\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\sinta\OneDrive\Lampiran\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\sinta\OneDrive\Lampiran\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\sinta\OneDrive\L

In [117]:
weekly_fact_enriched["interaction_intensity"] = (
    weekly_fact_enriched["plant_diversity_x_exposure"]
    + weekly_fact_enriched["task_diversity_x_exposure"]
    + weekly_fact_enriched["display_diversity_x_exposure"]
)

In [118]:
fragility_check = (
    weekly_fact_enriched
    .groupby(pd.qcut(
        weekly_fact_enriched["interaction_intensity"], 4
    ))
    .agg(
        mean_prediction=("predicted_hours", "mean"),
        mean_uncertainty=("prediction_uncertainty", "mean")
    )
    .reset_index()
)

fragility_check

C:\Users\sinta\AppData\Local\Temp\ipykernel_11024\2964610454.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(pd.qcut(


,interaction_intensity,mean_prediction,mean_uncertainty
0,"(5.999, 10.0]",2.002027,0.131187
1,"(10.0, 20.0]",2.436906,0.247579
2,"(20.0, 36.0]",2.531749,0.319524
3,"(36.0, 160.0]",4.924406,0.722912


| Pattern                           | Meaning             |
| --------------------------------- | ------------------- |
| Interaction ↑, Uncertainty ↑      | **Fragile zones**   |
| Interaction ↑, Uncertainty stable | High but manageable |
| Interaction ↓, Uncertainty ↓      | Stable operations   |


1. Low interaction (6–10)
- Predicted labor is low (~2.0)
- Model uncertainty is very low (~0.13)
- Stable, predictable operations. The model agrees strongly on outcomes.

2. Moderate interaction (10–20)
- Labor demand increases modestly (~2.44)
- Uncertainty nearly doubles (~0.25)
- Complexity begins to matter, but risk is still manageable.

3. High interaction (20–36)
- Labor rises slightly (~2.53)
- Uncertainty continues to climb (~0.32)
- Operations become sensitive; small changes can drive variability.

4. Extreme interaction (36–160)
- Labor demand nearly doubles (~4.92)
- Uncertainty spikes sharply (~0.72)
- This is a fragility regime: the model disagrees heavily, indicating unstable operating conditions.

#### D. PEAK ERROR (TAIL RISK) — RANDOM FOREST
Key Question:

When interactions are strong, how wrong can the model be in the worst cases?

#### 4.D.1 Compute 90th Percentile Absolute Error

In [119]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Predict on full dataset
rf_pred_full = rf_fragility.predict(
    weekly_fact_enriched[FEATURES]
)

weekly_fact_enriched["rf_prediction"] = rf_pred_full
weekly_fact_enriched["abs_error"] = np.abs(
    weekly_fact_enriched["weekly_hours"]
    - weekly_fact_enriched["rf_prediction"]
)

#### 4.D.2 Aggregate Peak Error by Interaction Intensity

In [120]:
peak_error_check = (
    weekly_fact_enriched
    .groupby(pd.qcut(
        weekly_fact_enriched["interaction_intensity"], 4
    ))
    .agg(
        mean_prediction=("rf_prediction", "mean"),
        mean_uncertainty=("prediction_uncertainty", "mean"),
        peak_90p_error=("abs_error", lambda x: np.percentile(x, 90))
    )
    .reset_index()
)

peak_error_check

C:\Users\sinta\AppData\Local\Temp\ipykernel_11024\477020613.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(pd.qcut(


,interaction_intensity,mean_prediction,mean_uncertainty,peak_90p_error
0,"(5.999, 10.0]",2.002027,0.131187,1.053349
1,"(10.0, 20.0]",2.436906,0.247579,1.046043
2,"(20.0, 36.0]",2.531749,0.319524,1.398136
3,"(36.0, 160.0]",4.924406,0.722912,1.725816


| Pattern                           | Meaning                 |
| --------------------------------- | ----------------------- |
| Interaction ↑, Peak Error ↑       | **Tail risk present**   |
| Uncertainty ↑ but Peak Error flat | No operational downside |
| Peak Error ↑ sharply              | **Danger zone**         |

1. **Low interaction (6–10)**

   * Low predicted labor (~2.0)
   * Very low uncertainty (~0.13)
   * Low peak error (~1.05)
     - **Stable and predictable**. Both average demand and worst-case errors are well contained.

2. **Moderate interaction (10–20)**

   * Labor increases modestly (~2.44)
   * Uncertainty nearly doubles (~0.25)
   * Peak error remains flat (~1.05)
     - **Manageable complexity**. Variability rises, but downside risk is still controlled.

3. **High interaction (20–36)**

   * Labor continues to rise (~2.53)
   * Uncertainty increases further (~0.32)
   * Peak error jumps (~1.40)
     - **Risk emerges**. Uncertainty now translates into larger worst-case misses.

4. **Extreme interaction (36–160)**

   * Labor demand nearly doubles (~4.92)
   * Uncertainty spikes (~0.72)
   * Peak error is highest (~1.73)
     - **Fragile, high-risk regime**. The model disagrees strongly *and* the cost of being wrong is large.

In [ ]:
# Normalize
weekly_fact_enriched["peak_error_norm"] = (
    weekly_fact_enriched["abs_error"]
    / weekly_fact_enriched["abs_error"].quantile(0.95)
).clip(upper=1.0)

In [ ]:
# FEATURE IMPORTANCE
fi = (
    pd.DataFrame({
        "feature": X_train.columns,
        "importance": rf.feature_importances_
    })
    .sort_values("importance", ascending=False)
)

fi.head(10)

,feature,importance
1,plant_diversity,0.676643
3,active_days,0.218077
4,dominant_season,0.053457
5,foot_traffic_score,0.034488
2,display_diversity,0.009780
0,task_diversity,0.007554


**A.] #1 Driver: historical_volatility	-> 0.32**

Labor demand is driven more by how unstable the workload is than by how busy it usually is.

Operational meaning:
- Places with fluctuating daily hours are hard to staff
- These areas need buffer capacity, not just average staffing

**B.] Labor Spikes Dominate by End-of-Plant Cycle Tasks**

Setup and teardown tasks cause sharper labor peaks than routine care.
- task_plant_removal (0.14)
- task_bulb_planting (0.13)
- task_poinsettia_swap (0.01)

Operational meaning:
- Seasonal transitions are the true labor risk
- Routine tasks (watering, deadheading) are predictable
- Swap/removal windows are where planning fails

**C.] Labor Demand Follows Short-Term Momentum and Seasonal**

lag_4w, lag_1w, lag_2w together ≈ 18%
- Last month matters more than last week alone
- Plan strategy should look 4 weeks ahead, not just react weekly

**D.] Daily Care Tasks Fine-Tune, Not Drive**

Watering, planting and media prep, disease control, deadheading ≈ secondary. These tasks adjust staffing, but rarely create labor peaks.
- No need to overreact to watering-heavy weeks (especially in summer)
- Focus planning on lifecycle shifts

**E.] Foot Traffic Matters BUT Indirectly**

- Foot_traffic_score (0.019) -> foot traffic amplifies labor when combined with complex tasks, not on its own.
- A high-traffic area (lobby) is fine during watering but at risk during plant removal or swap.

**F.] Certain Hotspots Consistenly Need More Labor, BUT Place Alone is Not Destiny**

- Hotspots: South Lobby, West Lobby, Main Hotel Area, Golf Club Area
- Don't staff purely by location
- Staff by location x task x season



**Bottomline**

Weekly labor demand is driven primarily by workload instability and plant lifecycle transition tasks, rather than average volume or location alone. Seasonal setup and teardown activities put the largest staffing risk, while routine maintenance tasks contribute predictable baseline demand. Location and foot traffic amplify labor needs only when combined with complex task mixes.

**Suggestion**

Labor planning should prioritize volatility and plant lifecycle transitions, not just busy locations or average workload.

### PHASE 5- Model Challanger: Gradient Boosting

- X → engineered features (task mix, seasonality, interaction intensity, etc.)
- y → target (weekly labor hours)
- Train / validation split already defined
- Metrics = MAE + RMSE (peak-sensitive)

IF Challenger has much lower Peak_MAE:

    → Use Challenger as risk lens

ELSE:
    → Keep RF as decision engine

Gradient Boosting was introduced strictly as a Challenger to stress-test the Random Forest Champion, with particular focus on peak workload sensitivity and interaction effects rather than average accuracy alone.

In [149]:
df_full = weekly_fact_enriched.copy()

In [150]:
interaction_cols = [
    "task_diversity",
    "plant_diversity",
    "display_diversity",
    "active_days"
]

for col in interaction_cols:
    df_full[f"{col}_x_exposure"] = (
        df_full[col] * df_full["foot_traffic_score"]
    )

Gradient Boosting Model

In [151]:
GB_FEATURES = [
    "task_diversity",
    "plant_diversity",
    "display_diversity",
    "active_days",
    "foot_traffic_score",
    "task_diversity_x_exposure",
    "plant_diversity_x_exposure",
    "display_diversity_x_exposure",
    "active_days_x_exposure"
]

In [154]:
# time-aware split
train_idx = df_full["week_id"] <= cutoff_week
val_idx   = df_full["week_id"] > cutoff_week

X_train_gb = df_full.loc[train_idx, GB_FEATURES]
y_train    = df_full.loc[train_idx, "weekly_hours"]

X_val_gb   = df_full.loc[val_idx, GB_FEATURES]
y_val      = df_full.loc[val_idx, "weekly_hours"]


In [155]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

gbr.fit(X_train_gb, y_train)

gbr_pred = gbr.predict(X_val_gb)

mae_gbr = mean_absolute_error(y_val, gbr_pred)
rmse_gbr = np.sqrt(mean_squared_error(y_val, gbr_pred))

mae_gbr, rmse_gbr


(0.6912722887035314, np.float64(1.031157507617494))

In [166]:
# sanity check first
assert len(gbr_pred) == len(y_val)

peak_mae_gbr = np.percentile(
    np.abs(y_val.values - gbr_pred),
    90
)

In [167]:
gbr_metrics = {
    "MAE": mae_gbr,
    "RMSE": rmse_gbr,
    "Peak_MAE": peak_mae_gbr
}

Rain Forest Model

In [176]:
[k for k in globals().keys() if k.startswith("rf")]

['rf_baseline',
 'rf_pred',
 'rf',
 'rf_stability',
 'rf_prediction_uncertainty',
 'rf_fragility',
 'rf_pred_full',
 'rf_A',
 'rf_B',
 'rf_metrics',
 'rf_model']

In [177]:
rf_model = rf_fragility

In [178]:
RF_FEATURES = [
    "task_diversity",
    "plant_diversity",
    "display_diversity",
    "active_days",
    "foot_traffic_score",
    "task_diversity_x_exposure",
    "plant_diversity_x_exposure",
    "display_diversity_x_exposure",
    "active_days_x_exposure"
]

In [179]:
X_val_rf = df_full.loc[val_idx, RF_FEATURES]
y_val_rf = df_full.loc[val_idx, "weekly_hours"]

assert list(X_val_rf.columns) == RF_FEATURES

In [180]:
rf_pred = rf_model.predict(X_val_rf)

import numpy as np

peak_mae_rf = np.percentile(
    np.abs(y_val_rf.values - rf_pred),
    90
)

In [181]:
rf_metrics = {
    "MAE": mae_rf,
    "RMSE": rmse_rf,
    "Peak_MAE": peak_mae_rf
}

In [182]:
# Champion–Challenger Scorecard
model_scorecard = pd.DataFrame([
    {
        "Model": "Random Forest",
        **rf_metrics
    },
    {
        "Model": "Gradient Boosting",
        **gbr_metrics
    }
])

model_scorecard = model_scorecard.round(3)
model_scorecard

,Model,MAE,RMSE,Peak_MAE
0,Random Forest,0.738,1.117,1.545
1,Gradient Boosting,0.691,1.031,1.802


| Use Case                              | Better Model          | Why                          |
| ------------------------------------- | --------------------- | ---------------------------- |
| Average weekly labor forecasting      | **Gradient Boosting** | Slightly lower general error |
| Peak demand / risk-sensitive planning | **Random Forest**     | Lower extreme-error exposure |
| Executive / ops safety bias           | **Random Forest**     | Fewer worst-case surprises   |
| Cost-optimization bias                | **Gradient Boosting** | Tighter mean predictions     |

---


**1. Average Accuracy (MAE & RMSE)**

* **Gradient Boosting performs better on average**

  * Lower **MAE** (0.691 vs 0.738)
  * Lower **RMSE** (1.031 vs 1.117)

**Meaning:**
On a *typical week*, Gradient Boosting predicts labor demand more precisely.
This makes it attractive for **efficiency optimization and fine-tuning schedules**.

---
**2. Tail Risk (Peak_MAE)**

* **Random Forest has a lower Peak_MAE**

  * RF: **1.545**
  * GB: **1.802**

**Meaning:**
In the **worst 10% of weeks**, Gradient Boosting makes **larger mistakes** than Random Forest.

This is critical because:

* Peak weeks drive **overtime, guest impact, and budget overruns**
* Large misses are operationally more damaging than small average errors

---

**3. Risk vs Efficiency Trade-off (Key Insight)**

> **Gradient Boosting is more accurate on average, but less safe under stress.**
> **Random Forest is slightly less precise, but more reliable in worst-case scenarios.**
---

**Recommended Champion–Challenger Decision**

**Champion: Random Forest**

Use when:

* Planning staffing buffers
* Managing high-interaction / Silent Drain zones
* Protecting guest experience during seasonal transitions

**Challenger: Gradient Boosting**

Use when:

* Optimizing labor in stable / Efficient Staple zones
* Fine-tuning schedules where volatility is low
* Seeking incremental efficiency gains

---

**Bottomline**

> *“Gradient Boosting improves average accuracy, but Random Forest provides stronger protection against worst-case labor underestimation. For operational planning, risk control outweighs marginal accuracy gains.”*

---

#### Quantile Gradient Boosting

In [263]:
from sklearn.ensemble import GradientBoostingRegressor

quantiles = [0.1, 0.5, 0.9]
quantile_models = {}

for q in quantiles:
    gbr_q = GradientBoostingRegressor(
        loss="quantile",
        alpha=q,
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3,
        subsample=0.8,
        random_state=42
    )
    gbr_q.fit(X_train, y_train)
    quantile_models[q] = gbr_q


In [264]:
quantile_preds = pd.DataFrame(
    index=X_val.index
)

quantile_preds["P10"] = quantile_models[0.1].predict(X_val)
quantile_preds["P50"] = quantile_models[0.5].predict(X_val)
quantile_preds["P90"] = quantile_models[0.9].predict(X_val)


In [266]:
quantile_results = weekly_ops.iloc[val_idx][
    ["week", "place_name_id"]
].copy()

quantile_results = quantile_results.join(quantile_preds)
quantile_results["actual"] = y_val.values


In [267]:
quantile_results["uncertainty_width"] = (
    quantile_results["P90"] - quantile_results["P10"]
)

quantile_results["miss_outside_band"] = (
    (quantile_results["actual"] < quantile_results["P10"]) |
    (quantile_results["actual"] > quantile_results["P90"])
)


In [268]:
quantile_results["miss_outside_band"].mean()


np.float64(0.25190839694656486)

In [276]:
champion_model = gbr

### PHASE 6- Risk & Priority Layer

In [285]:
op_fact["date"] = pd.to_datetime(op_fact["date"])

op_fact["week"] = (
    op_fact["date"]
    .dt.to_period("W")
    .apply(lambda r: r.start_time)
)


In [ ]:
df_model = (
    op_fact
    .merge(area_dim, on="place_name_id", how="left")
    .groupby(["place_name", "week"], as_index=False)
    .agg(
        weekly_labor_hours=("hours", "sum"),
        avg_hours_per_task=("hours", "mean"),
        task_frequency_per_week=("task_id", "count"),
        seasonality_variance=("hours", "std")
    )
)


In [295]:
df_model = df_model.sort_values(["place_name", "week"])

df_model["lag_1_hours"] = (
    df_model
    .groupby("place_name")["weekly_labor_hours"]
    .shift(1)
)

df_model["rolling_4w_std"] = (
    df_model
    .groupby("place_name")["weekly_labor_hours"]
    .rolling(4)
    .std()
    .reset_index(level=0, drop=True)
)


In [296]:
TARGET = "weekly_labor_hours"
TIME_COL = "week"

FEATURES = [
    "avg_hours_per_task",
    "task_frequency_per_week",
    "seasonality_variance",
    "lag_1_hours",
    "rolling_4w_std"
]

In [318]:
split_week = df_model[TIME_COL].quantile(0.8)

train_df = df_model[df_model[TIME_COL] <= split_week]
val_df   = df_model[df_model[TIME_COL] > split_week]

X_train = train_df[FEATURES]
y_train = train_df[TARGET]

X_val   = val_df[FEATURES]
y_val   = val_df[TARGET]


In [319]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed   = imputer.transform(X_val)


In [320]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor

pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("model", GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        random_state=42
    ))
])


In [321]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.05, max_depth=4,
                                           n_estimators=300,
                                           random_state=42))])

In [322]:
val_pred = pipeline.predict(X_val)


In [323]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae  = mean_absolute_error(y_val, val_pred)
rmse = np.sqrt(mean_squared_error(y_val, val_pred))

mae, rmse


(0.2987297609584165, np.float64(1.3516067781894996))

In [324]:
def peak_mae(y_true, y_pred, q=0.9):
    threshold = np.quantile(y_true, q)
    mask = y_true >= threshold
    return mean_absolute_error(y_true[mask], y_pred[mask])

peak_error = peak_mae(y_val, val_pred)
peak_error


1.7005180764791605

In [326]:
trained_gbr = pipeline.named_steps["model"]

In [327]:
feature_importance = (
    pd.DataFrame({
        "feature": X_train.columns,
        "importance": trained_gbr.feature_importances_
    })
    .sort_values("importance", ascending=False)
)

feature_importance


,feature,importance
1,task_frequency_per_week,0.826739
0,avg_hours_per_task,0.149712
4,rolling_4w_std,0.015819
2,seasonality_variance,0.007239
3,lag_1_hours,0.000491


In [330]:
# FORECAST NEXT WEEK
latest_week = df_model["week"].max()

future_df = (
    df_model[df_model["week"] == latest_week]
    .copy()
)

X_future = future_df[FEATURES]

future_df["forecast_labor_hours"] = pipeline.predict(X_future)


In [332]:
forecast_output = future_df[
    [
        "place_name",
        "week",
        "forecast_labor_hours",
        "lag_1_hours",
        "rolling_4w_std"
    ]
]

forecast_output


,place_name,week,forecast_labor_hours,lag_1_hours,rolling_4w_std
73,Golf Club Area,2025-04-14,13.660973,4.68,7.843358
115,Golf Club Dining,2025-04-14,3.330421,10.50,3.185608
165,Guest Room,2025-04-14,4.124097,16.50,13.566642
232,La Taverne,2025-04-14,6.773809,1.50,3.081941
296,Main Hotel Area,2025-04-14,3.508880,4.50,1.471176
345,Main Lobby,2025-04-14,4.475425,4.50,4.787507
377,Mezzanine,2025-04-14,2.632061,1.50,0.688253
446,Outdoor Pool,2025-04-14,5.443461,5.07,6.482841
488,Pickleball Court,2025-04-14,5.911495,3.48,4.806329
531,Racket Court,2025-04-14,8.508996,12.50,3.697039


### PHASE 7- Decision Logic

In [333]:
import numpy as np

# Safety: ensure columns exist
required = ["forecast_labor_hours", "lag_1_hours", "rolling_4w_std"]
missing = [c for c in required if c not in future_df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

# Peak threshold based on historical distribution
peak_threshold = df_model["weekly_labor_hours"].quantile(0.9)

future_df["peak_risk_flag"] = future_df["forecast_labor_hours"] >= peak_threshold

# Volatility score (0–1)
future_df["volatility_score"] = (
    future_df["rolling_4w_std"] / df_model["rolling_4w_std"].max()
).clip(0, 1)


In [334]:
future_df["priority_tier"] = pd.cut(
    future_df["forecast_labor_hours"],
    bins=[0, 6, 10, 15, np.inf],
    labels=["Low", "Medium", "High", "Critical"],
    include_lowest=True
)


In [335]:
# Staffing Rules 
def staffing_action(hours, peak, vol):
    if peak and hours >= 15:
        return "Add extra crew"
    if hours >= 10 and vol >= 0.6:
        return "Pre-allocate backup staff"
    if hours >= 10:
        return "Maintain staffing"
    return "Reduce / Reallocate"

future_df["recommended_action"] = future_df.apply(
    lambda x: staffing_action(
        x["forecast_labor_hours"],
        x["peak_risk_flag"],
        x["volatility_score"]
    ),
    axis=1
)


In [337]:
decision_output = future_df[[
    "place_name",
    "week",
    "forecast_labor_hours",
    "volatility_score",
    "peak_risk_flag",
    "priority_tier",
    "recommended_action"
]].sort_values(["priority_tier", "forecast_labor_hours"], ascending=[True, False])

decision_output


,place_name,week,forecast_labor_hours,volatility_score,peak_risk_flag,priority_tier,recommended_action
488,Pickleball Court,2025-04-14,5.911495,0.301552,False,Low,Reduce / Reallocate
446,Outdoor Pool,2025-04-14,5.443461,0.406738,False,Low,Reduce / Reallocate
345,Main Lobby,2025-04-14,4.475425,0.300372,False,Low,Reduce / Reallocate
165,Guest Room,2025-04-14,4.124097,0.851181,False,Low,Reduce / Reallocate
744,West Hotel Area,2025-04-14,4.056920,0.080117,False,Low,Reduce / Reallocate
296,Main Hotel Area,2025-04-14,3.508880,0.092303,False,Low,Reduce / Reallocate
115,Golf Club Dining,2025-04-14,3.330421,0.199867,False,Low,Reduce / Reallocate
706,The Hotel Bar,2025-04-14,3.004417,0.200326,False,Low,Reduce / Reallocate
377,Mezzanine,2025-04-14,2.632061,0.043181,False,Low,Reduce / Reallocate
531,Racket Court,2025-04-14,8.508996,0.231955,False,Medium,Reduce / Reallocate


### PHASE 8- Labor Hour ML Based Forecasting

Multi Week Forecasting

In [339]:
HORIZON_WEEKS = 12 # change as needed

In [341]:
last_week = df_model["week"].max()

history_df = (
    df_model
    .sort_values(["place_name", "week"])
    .groupby("place_name")
    .tail(4)        # keep last 4 weeks per place name for rolling stats
    .copy()
)

In [343]:
# Rolling Forecast Loop
future_forecasts = []

current_history = history_df.copy()

for step in range(1, HORIZON_WEEKS + 1):

    forecast_week = last_week + pd.Timedelta(weeks=step)

    # Build features for next week
    next_df = (
        current_history
        .groupby("place_name")
        .apply(lambda x: pd.Series({
            "place_name": x["place_name"].iloc[-1],
            "week": forecast_week,
            "avg_hours_per_task": x["avg_hours_per_task"].mean(),
            "task_frequency_per_week": x["task_frequency_per_week"].mean(),
            "seasonality_variance": x["seasonality_variance"].std(),
            "lag_1_hours": x["weekly_labor_hours"].iloc[-1],
            "rolling_4w_std": x["weekly_labor_hours"].tail(4).std()
        }))
        .reset_index(drop=True)
    )

    # Predict
    X_next = next_df[FEATURES]
    next_df["forecast_labor_hours"] = pipeline.predict(X_next)

    # Store result
    future_forecasts.append(next_df)

    # Append forecast back into history (THIS IS THE ROLLING PART)
    feedback = next_df.rename(
        columns={"forecast_labor_hours": "weekly_labor_hours"}
    )

    current_history = pd.concat(
        [current_history, feedback],
        ignore_index=True
    )


C:\Users\sinta\AppData\Local\Temp\ipykernel_26656\1214847584.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\sinta\AppData\Local\Temp\ipykernel_26656\1214847584.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\sinta\AppData\Local\Temp\ipykernel_26656\1214847584.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on t

In [344]:
forecast_multiweek = pd.concat(future_forecasts, ignore_index=True)

forecast_multiweek


,place_name,week,avg_hours_per_task,task_frequency_per_week,seasonality_variance,lag_1_hours,rolling_4w_std,forecast_labor_hours
0,Golden Bee,2025-04-21,1.375000,2.50,0.000000,3.000000,1.732051,2.720501
1,Golf Club Area,2025-04-21,1.403333,7.00,0.134069,14.000000,7.843358,9.046331
2,Golf Club Dining,2025-04-21,1.351667,4.50,0.250000,3.420000,3.185608,5.368432
3,Guest Room,2025-04-21,1.531568,10.00,0.625642,4.240000,13.566642,15.988864
4,Indoor Pool,2025-04-21,1.243750,1.25,NaN,1.000000,0.758172,1.267772
...,...,...,...,...,...,...,...,...
259,South Lobby,2025-07-07,1.494000,8.25,0.084909,11.887588,0.045088,11.887588
260,Tennis Court,2025-07-07,1.229167,3.50,0.083638,3.713804,0.000000,3.713804
261,The Hotel Bar,2025-07-07,1.051250,2.50,0.199337,2.130584,0.000000,2.099978
262,West Hotel Area,2025-07-07,1.397917,2.00,0.273714,2.836829,0.001382,2.836829


In [345]:
# Risk & Priority
peak_threshold = df_model["weekly_labor_hours"].quantile(0.9)

forecast_multiweek["peak_risk_flag"] = (
    forecast_multiweek["forecast_labor_hours"] >= peak_threshold
)

forecast_multiweek["priority_tier"] = pd.cut(
    forecast_multiweek["forecast_labor_hours"],
    bins=[0, 6, 10, 15, np.inf],
    labels=["Low", "Medium", "High", "Critical"],
    include_lowest=True
)


In [347]:
multiweek_output = forecast_multiweek[[
    "place_name",
    "week",
    "forecast_labor_hours",
    "rolling_4w_std",
    "peak_risk_flag",
    "priority_tier"
]]

multiweek_output


,place_name,week,forecast_labor_hours,rolling_4w_std,peak_risk_flag,priority_tier
0,Golden Bee,2025-04-21,2.720501,1.732051,False,Low
1,Golf Club Area,2025-04-21,9.046331,7.843358,False,Medium
2,Golf Club Dining,2025-04-21,5.368432,3.185608,False,Low
3,Guest Room,2025-04-21,15.988864,13.566642,True,Critical
4,Indoor Pool,2025-04-21,1.267772,0.758172,False,Low
...,...,...,...,...,...,...
259,South Lobby,2025-07-07,11.887588,0.045088,False,High
260,Tennis Court,2025-07-07,3.713804,0.000000,False,Low
261,The Hotel Bar,2025-07-07,2.099978,0.000000,False,Low
262,West Hotel Area,2025-07-07,2.836829,0.001382,False,Low


In [357]:
multiweek_output.to_csv(
    "labor_multiweek_forecast.csv",
    index=False
)

Rolling-horizon forecasting, where each week’s prediction feeds into the next, mirroring real operational uncertainty.

#### Confidence Bands (P10-P90)

P10 → optimistic (best-case)

P50 → baseline forecast

P90 → conservative (worst-case)

In [348]:
# Residuals = actual - predicted
residuals = y_val - val_pred

In [349]:
p10_error = residuals.quantile(0.10)
p50_error = residuals.quantile(0.50)  # usually ~0
p90_error = residuals.quantile(0.90)

p10_error, p50_error, p90_error

(np.float64(-0.13010440895087144),
 np.float64(0.002612535225375967),
 np.float64(0.2203270776123077))

Interpretation:

Negative → model tends to overpredict

Positive → model tends to underpredict

In [351]:
# APPLY CONFIDENCE BANDS TO MULTI-WEEK FORECAST
forecast_multiweek["forecast_p10"] = (
    forecast_multiweek["forecast_labor_hours"] + p10_error
)

forecast_multiweek["forecast_p50"] = (
    forecast_multiweek["forecast_labor_hours"] + p50_error
)

forecast_multiweek["forecast_p90"] = (
    forecast_multiweek["forecast_labor_hours"] + p90_error
)

In [352]:
for c in ["forecast_p10", "forecast_p50", "forecast_p90"]:
    forecast_multiweek[c] = forecast_multiweek[c].clip(lower=0)


In [354]:
# ADD RISK FLAGS
# Conservative staffing signal
forecast_multiweek["high_risk_band"] = (
    forecast_multiweek["forecast_p90"] >= peak_threshold
)

In [355]:
confidence_output = forecast_multiweek[[
    "place_name",
    "week",
    "forecast_p10",
    "forecast_p50",
    "forecast_p90",
    "priority_tier",
    "high_risk_band"
]]

confidence_output


,place_name,week,forecast_p10,forecast_p50,forecast_p90,priority_tier,high_risk_band
0,Golden Bee,2025-04-21,2.590396,2.723113,2.940828,Low,False
1,Golf Club Area,2025-04-21,8.916227,9.048944,9.266659,Medium,False
2,Golf Club Dining,2025-04-21,5.238328,5.371045,5.588759,Low,False
3,Guest Room,2025-04-21,15.858760,15.991477,16.209191,Critical,True
4,Indoor Pool,2025-04-21,1.137667,1.270384,1.488099,Low,False
...,...,...,...,...,...,...,...
259,South Lobby,2025-07-07,11.757484,11.890200,12.107915,High,False
260,Tennis Court,2025-07-07,3.583700,3.716417,3.934131,Low,False
261,The Hotel Bar,2025-07-07,1.969874,2.102591,2.320305,Low,False
262,West Hotel Area,2025-07-07,2.706725,2.839442,3.057157,Low,False


In [356]:
confidence_output.to_csv(
    "labor_forecast_confidence.csv",
    index=False
)

Explanation:

- place_name:        Golden Bee

- week:              2025-04-21

- forecast_p10:      2.59     -> Min. staffing scenario (~2.6 labor hours this week)

- forecast_p50:      2.72     -> Baseline

- forecast_p90:      2.94     -> Safety Buffer

- priority_tier:     Low      -> Doesn’t compete for crew time

- high_risk_band:    False -> No overtime risk, No emergency staffing, No reallocation needed


Further Improvements:
- Add exogenous features (weather, events)
- Retrain monthly
- Monitor drift
- Compare with naïve baseline



### PHASE 9- Weekly Labor Demand & Priority Outlook

##### ML Based Forecasting Output Summary 

This forecasting output presents **weekly labor demand projections by location**, enriched with **uncertainty bands and operational priority signals**.
Each record represents a forward-looking view of expected labor hours, allowing managers to plan staffing with both **confidence and risk awareness**.

##### What the forecast shows

* **P50 (Median forecast)** represents the most likely weekly labor requirement.
* **P10–P90 range** captures uncertainty, indicating best-case to high-stress scenarios.
* **Priority tiers** translate forecasted workload into clear operational focus levels.
* **High-risk band flags** identify areas where elevated demand and uncertainty coincide.

---

##### Key insights from the output

**1. Labor demand is highly uneven across locations**
Weekly median labor forecasts range from **~1–3 hours in low-intensity areas** (e.g., Indoor Pool, Hotel Bar) to **~12–16 hours in high-impact zones** such as Guest Rooms and major lobbies.
This confirms that labor needs are **structurally concentrated**, not evenly distributed.

**2. Critical areas show both high demand and elevated risk**
Locations classified as **Critical** (e.g., Guest Room areas) consistently exhibit:

* The highest median labor demand (≈ **16 hours/week**)
* Upper-bound forecasts exceeding **16 hours**
* Active **high-risk band flags**

These areas represent the **greatest risk of service degradation** if labor is underestimated.

**3. High-priority zones require proactive buffering, not reaction**
Areas labeled **High** or **Medium** priority (e.g., Golf Club Area, South Lobby, West Lobby) show:

* Sustained weekly demand in the **9–12 hour range**
* Relatively tight uncertainty bands, indicating predictable but consistently heavy workload

These zones benefit most from **planned staffing buffers**, not ad-hoc adjustments.

**4. Most locations remain low-risk and stable**
The majority of locations fall into the **Low priority tier**, with:

* Median weekly labor below **~4 hours**
* Narrow uncertainty ranges
* No high-risk band activation

This creates an opportunity to **avoid overstaffing** and reallocate attention to higher-impact zones.

---

##### Operational value of this forecast

This output enables teams to:

* Plan weekly staffing based on **expected demand, not last-week activity**
* Allocate buffers selectively to **high-risk and critical locations**
* Justify labor decisions using **quantified uncertainty**, not intuition
* Protect guest-facing areas where underestimation has the highest impact

---

##### Bottom-line interpretation

> This forecast transforms labor planning from a uniform schedule into a **location-specific, risk-aware staffing strategy**, ensuring that attention and resources are focused where operational failure would matter most.

##### How to Use This Forecast Table (Operational Guide)

This table is designed for **weekly staffing and supervision planning**, not retrospective analysis.

##### Step 1 — Anchor on the P50 (Median Forecast)

* Use **`forecast_p50`** as the **baseline weekly labor plan** for each location.
* This represents the most likely labor requirement under normal conditions.

> Example:
> *Guest Room – P50 ≈ 16 hours* → Plan ~16 hours as the starting point.

---

##### Step 2 — Use P90 as a Buffer Indicator

* **`forecast_p90`** represents a **high-stress scenario**, not an expectation.
* The gap between P50 and P90 indicates how much **buffer capacity** may be required.

> Tight range → stable workload
> Wide range → higher uncertainty → consider buffer

---

##### Step 3 — Follow Priority Tier, Not Just Hours

* **Priority tier** determines *attention level*, not just staffing volume.
* Two locations with similar hours may require **different actions** if their priority differs.

---

##### Step 4 — Act Immediately on High-Risk Band = TRUE

* If **`high_risk_band = TRUE`**, do not rely solely on P50.
* These locations should be reviewed for:

  * Staffing buffer
  * Supervisor coverage
  * Task deferral risk

---

##### Step 5 — Reallocate from Low Priority Zones

* Locations with **Low priority + narrow uncertainty** are candidates for:

  * Reduced buffers
  * Flexible staffing
  * Labor reallocation during peak weeks

---

##### Decision Playbook by Priority Tier

##### 🟢 **Low Priority**

**Typical profile**

* Median demand: ~1–4 hours/week
* Narrow P10–P90 range
* High-risk band: False

**Action**

* Staff to **P50 only**
* No buffer required
* Use as a **flex pool** if other areas spike

**Management focus**

* Minimal supervision
* Routine maintenance only

---

##### 🟡 **Medium Priority**

**Typical profile**

* Median demand: ~6–9 hours/week
* Predictable but sustained workload
* High-risk band: False

**Action**

* Staff to **P50**
* Add **light buffer** during seasonal transitions
* Monitor week-over-week trend

**Management focus**

* Weekly check-in
* Prevent gradual backlog buildup

---

##### 🟠 **High Priority**

**Typical profile**

* Median demand: ~10–12 hours/week
* Guest-facing or high-traffic areas
* Moderate uncertainty

**Action**

* Staff between **P50 and P90**
* Pre-assign backup labor
* Avoid task deferral

**Management focus**

* Active supervision
* Prioritize over Medium/Low zones

---

##### 🔴 **Critical Priority**

**Typical profile**

* Median demand: ≥15 hours/week
* High guest exposure
* High-risk band often TRUE

**Action**

* Staff **at or near P90**
* Lock in buffer hours
* Do not reallocate labor away from these zones

**Management focus**

* Daily visibility
* Zero tolerance for understaffing
* Protect guest experience first

---

##### Final Operating Principle

> **Not all hours are equal.**
> This forecast ensures labor is allocated based on **risk, impact, and predictability**, not just total workload.


### PHASE 10- Model Deployment

In [13]:
model.feature_names_in_

array(['task_diversity', 'plant_diversity', 'display_diversity',
       'active_days', 'foot_traffic_score', 'task_diversity_x_exposure',
       'plant_diversity_x_exposure', 'display_diversity_x_exposure',
       'active_days_x_exposure'], dtype=object)

In [1]:
import joblib

model = joblib.load("labor_demand_model.pkl")


In [44]:
import pandas as pd

# 1️⃣ RAW INPUT (authoritative source)
input_raw = pd.DataFrame([{
    "task_diversity": 4,
    "plant_diversity": 6,
    "display_diversity": 3,
    "active_days": 5,
    "foot_traffic_score": 0.8
}])

# 2️⃣ BUILD FEATURES
input_data = input_raw.copy()

input_data["task_diversity_x_exposure"] = (
    input_data["task_diversity"] * input_data["foot_traffic_score"]
)
input_data["plant_diversity_x_exposure"] = (
    input_data["plant_diversity"] * input_data["foot_traffic_score"]
)
input_data["display_diversity_x_exposure"] = (
    input_data["display_diversity"] * input_data["foot_traffic_score"]
)
input_data["active_days_x_exposure"] = (
    input_data["active_days"] * input_data["foot_traffic_score"]
)

# 3️⃣ ENFORCE FEATURE ORDER
input_data = input_data[model.feature_names_in_]

# 4️⃣ SANITY CHECK
print("ACTIVE DAYS USED:", input_data["active_days"].iloc[0])

# 5️⃣ PREDICT
per_day = model.predict(input_data)[0]
weekly = per_day * input_data["active_days"].iloc[0]

print(f"Per-day hours: {per_day:.2f}")
print(f"Weekly hours: {weekly:.2f}")


ACTIVE DAYS USED: 5
Per-day hours: 6.76
Weekly hours: 33.80


#### Business Interpretation of the Output:
Active days used: 5  
Predicted labor hours per active day: 6.76  
Predicted total weekly labor hours: 33.8


🧠 Business Terms
This forecast indicates that the operational workload for this location is below the standard 40-hour weekly staffing assumption for the upcoming week.

Based on:
- current foot traffic levels,
- task and plant diversity,
- and historical labor patterns,

the model estimates that the location requires approximately 6.8 hours of labor per day, resulting in a total weekly demand of ~34 hours across five active days.


💼 Operational Insight
Compared to a typical 40-hour baseline allocation, this suggests:
- ~6 hours of excess capacity for the week
- An opportunity to reallocate labor to higher-risk or higher-exposure areas
- Reduced likelihood of overtime or last-minute staffing adjustments

This does not imply cutting staff, but rather optimizing deployment based on predicted demand.

💰 Financial Implication
If average labor cost is $20–25 per hour, this efficiency represents:
- $120–150 in weekly labor capacity
- ~$6,000–7,500 annually per location, if consistently observed
These savings can be redirected toward:
- peak-season coverage,
- guest-facing improvements,
- contingency staffing during high-risk periods.